# data link layer with scapy wrapper 

> send or receive a CAN message to a bus, ISO11898-1
> using the python-can library
> testing on vcan interface

In [ ]:
#| default_exp data_link_scapy

In [ ]:
#| hide
from __future__ import annotations
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| export
# from multiprocessing import Process, Event
# from multiprocessing import synchronize, Manager
# from multiprocessing.managers import DictProxy
from pprint import pprint, PrettyPrinter
import subprocess
import threading
from datetime import datetime
import json
import argparse
import os
import git
import sys
import signal
import time
from multiprocessing import Manager
from multiprocessing.managers import DictProxy

In [ ]:
#| export
import can
# from can.interface import Bus
# from can import Message 
import cantools
from cantools.database import Message as MessageTpl
from cantools.database.can.database import Database

In [ ]:
#| export
pp = PrettyPrinter(indent=4, width=80, compact=True)

In [ ]:
#| export 
repo = git.Repo("./", search_parent_directories=True)  # get the Repo object of tspace
if os.path.basename(repo.working_dir) != "candycan":  # I'm in the parent repo!
    repo = repo.submodule("candycan").module()
pprint(repo.working_dir)

'/home/n/devel/anting_remote/eos/tspace/candycan'


In [ ]:
#| export
def get_argparser() -> argparse.ArgumentParser:
    """_summary_ get CAN bus, dbc config and the message to send

    Returns:
        argparse.ArgumentParser: _description_
    """

    parser = argparse.ArgumentParser("Get the CAN Bus channel, bitrate and dbc path")

    parser.add_argument(
        "-t",
        "--type",
        type=str,
        default="socketcan",
        help="The type of the CAN bus",
    )

    parser.add_argument(
        "-c",
        "--channel",
        type=str,
        default="vcan1",
        help="The CAN bus channel to connect to",
    )

    parser.add_argument(
        "-b", "--bitrate", type=int, default=250000, help="The bitrate of the CAN bus"
    )

    parser.add_argument(
        "-d",
        "--dbc",
        type=str,
        default="../../res/motohawk_new.dbc",
        help="The path to the dbc file",
    )

    parser.add_argument(
        "-m",
        "--message",
        type=str,
        default="ExampleMessage",
        help="The message to send",
    )

    parser.add_argument(
        "-e",
        "--extended",
        action="store_true",
        help="If the arbitration id is extended",
    )

    return parser


In [ ]:
#| export
done = threading.Event()
def signal_usr1(signum, frame):
    """Handle USR1 signal as an event to set the received flag."""
    # global received
    # received = True

    done.set()
    # print("received signal, sending done!")

In [ ]:
#| export
def send_msg(db:Database, message:str, payload:bytes, channel:str, bitrate:int, bus_type: str, is_extended: bool) -> None:

    message_definition = db.get_message_by_name(message)
    data_dict = json.loads(payload.decode())

    # print("Prepare sending message")
    sys.stdout.flush()
    can_data = message_definition.encode(data_dict)
    message = can.Message(
        arbitration_id=message_definition.frame_id,
        data=can_data,
        is_extended_id=is_extended,
    )
    # print(message)
    # print("Before sending message")
    with can.interface.Bus(bustype=bus_type, channel=channel, bitrate=bitrate) as bus:
        bus.send(message)


In [ ]:
db_can = cantools.database.load_file(repo.working_dir+'/res/motohawk_new.dbc')
db_can.messages


[message('ExampleMessage', 0x1f0, False, 8, {None: 'Example message used as template in MotoHawk models.'}),
 message('NewMessage', 0x254, False, 8, {None: 'self made message'})]

In [ ]:
example_message: MessageTpl = db_can.get_message_by_name('ExampleMessage')
pprint(example_message.signals)
pprint(example_message.__dict__)
example_message.frame_id

[signal('Enable', 7, 1, 'big_endian', False, None, 1, 0, None, None, '-', False, None, {0: 'Disabled', 1: 'Enabled'}, None, None),
 signal('AverageRadius', 6, 6, 'big_endian', False, None, 0.1, 0, 0, 5, 'm', False, None, None, None, None),
 signal('Temperature', 0, 12, 'big_endian', True, None, 0.01, 250, 229.52, 270.47, 'degK', False, None, None, None, None)]
{'_autosar': None,
 '_bus_name': None,
 '_codecs': {'formats': Formats(big_endian=<bitstruct.c.CompiledFormatDict object>, little_endian=<bitstruct.c.CompiledFormatDict object>, padding_mask=35184372088831),
             'multiplexers': {},
             'signals': [signal('Enable', 7, 1, 'big_endian', False, None, 1, 0, None, None, '-', False, None, {0: 'Disabled', 1: 'Enabled'}, None, None),
                         signal('AverageRadius', 6, 6, 'big_endian', False, None, 0.1, 0, 0, 5, 'm', False, None, None, None, None),
                         signal('Temperature', 0, 12, 'big_endian', True, None, 0.01, 250, 229.52, 270.47, '

496

In [ ]:
# install vcan interface with encrypted password to sudo 
os.system("gpg -d -q ~/.sshpasswd.gpg | sshpass -v sudo modprobe vcan")
# sshpass -v -p asdf sudo ip link add dev vcan0 type vcan
os.system("gpg -d -q ~/.sshpasswd.gpg | sshpass -v sudo ip link add dev vcan0 type vcan")
os.system("ip link show vcan0")
# !gpg -d -q ~/.sshpasswd.gpg | sshpass -v sudo ip link set vcan0 type vcan bitrate 500000  # vcan does not support set bitrate on command line!
# !sshpass -p asdf sudo ip link add dev vcan0 type vcan
os.system("gpg -d -q ~/.sshpasswd.gpg | sshpass -v sudo ip link set up vcan0")
# !sshpass -v -p asdf sudo ip link set up vcan0

SSHPASS: searching for password prompt using match "assword"
SSHPASS: read: [sudo] password for n: 
SSHPASS: detected prompt. Sending password.


SSHPASS: read: 



0

SSHPASS: searching for password prompt using match "assword"
SSHPASS: read: [sudo] password for n: 
SSHPASS: detected prompt. Sending password.
SSHPASS: read: 



0

16: vcan0: <NOARP> mtu 72 qdisc noop state DOWN mode DEFAULT group default qlen 1000
    link/can 


0

SSHPASS: searching for password prompt using match "assword"
SSHPASS: read: [sudo] password for n: 
SSHPASS: detected prompt. Sending password.
SSHPASS: read: 



0

In [ ]:
# # install vcan interface with encrypted password to sudo 
# !gpg -d -q ~/.sshpasswd.gpg | sshpass -v sudo modprobe vcan
# # sshpass -v -p asdf sudo ip link add dev vcan0 type vcan
# !gpg -d -q ~/.sshpasswd.gpg | sshpass -v sudo ip link add dev vcan0 type vcan
# !ip link show vcan0
# # !gpg -d -q ~/.sshpasswd.gpg | sshpass -v sudo ip link set vcan0 type vcan bitrate 500000  # vcan does not support set bitrate on command line!
# # !sshpass -p asdf sudo ip link add dev vcan0 type vcan
# !gpg -d -q ~/.sshpasswd.gpg | sshpass -v sudo ip link set up vcan0
# # !sshpass -v -p asdf sudo ip link set up vcan0

In [ ]:
data_dict ={'Temperature': 250.1, 'AverageRadius': 3.2, 'Enable': 1}
data_json_bytes = json.dumps(data_dict).encode('utf-8')
data_json_bytes
json.loads(data_json_bytes.decode())

can_data = example_message.encode({'Temperature': 250.1, 'AverageRadius': 3.2, 'Enable': 1})
example_message.decode(can_data)

b'{"Temperature": 250.1, "AverageRadius": 3.2, "Enable": 1}'

{'Temperature': 250.1, 'AverageRadius': 3.2, 'Enable': 1}

{'Enable': 'Enabled', 'AverageRadius': 3.2, 'Temperature': 250.1}

In [ ]:
bus = can.interface.Bus(bustype='socketcan', channel='vcan0', bitrate=250000)
message_to_send = can.Message(arbitration_id=example_message.frame_id, data=can_data, is_extended_id=False) 	
# can_bus.send(message)

In [ ]:
manager = Manager()
message_proxy = manager.dict()

In [ ]:
#| export
def receive_message(message_proxy: DictProxy,bus: can.interface.Bus)->None:
	print('waiting for message')
	msg:can.Message = bus.recv()
	print('message received')
	message_proxy['timestamp'] = msg.timestamp
	message_proxy['arbitration_id'] = msg.arbitration_id
	message_proxy['data']=msg.data

In [ ]:
proc = subprocess.Popen(
	['python', 
        repo.working_dir+'/candycan/data_link_scapy.py', 
		'-t', 'socketcan', 
		'-c' , 'vcan0', 
		'-b', '25000',
		'-d', repo.working_dir+'/res/motohawk_new.dbc',
		'-m', 'ExampleMessage',
		], 
	stdout=subprocess.PIPE, 
	stderr=subprocess.PIPE, 
	stdin=subprocess.PIPE
	)
print(f'PARENT: {proc.pid} before signaling child')


PARENT: 357230 before signaling child


In [ ]:
data_json_bytes
data_json_bytes.decode()
json.loads(data_json_bytes.decode())

b'{"Temperature": 250.1, "AverageRadius": 3.2, "Enable": 1}'

'{"Temperature": 250.1, "AverageRadius": 3.2, "Enable": 1}'

{'Temperature': 250.1, 'AverageRadius': 3.2, 'Enable': 1}

In [ ]:
try:
	outs, errs = proc.communicate(data_json_bytes, timeout=1)
except subprocess.TimeoutExpired:
	print(f'PARENT: {proc.pid}; TimeoutExpired')
	# outs, errs = proc.communicate()
	# print(f'PARENT: {proc.pid}; outs: {outs}; errs: {errs} TimeoutExpired')
# sys.stdout.flush()
# time.sleep(1)


In [ ]:

receive_message(message_proxy, bus)
# proc_receive = subprocess.Popen(
# 	['python', 
#         '../candycan/receive_message.py', 
# 		'-t', 'socketcan', 
# 		'-c' , 'vcan0', 
# 		'-b', '25000',
# 		'-d', '../res/motohawk_new.dbc',
# 		'-m', 'ExampleMessage',
# 		], 
# 	# stdout=subprocess.PIPE, 
# 	# stderr=subprocess.PIPE, 
# 	# stdin=subprocess.PIPE
# 	)
# print(f'PARENT: {proc.pid} before signaling child')

# stdout_raw, stderr_raw = proc_receive.communicate()
# stdout_value = stdout_raw.decode('utf-8')
# stderr_value = stderr_raw.decode('utf-8')
# # [0].decode('utf-8')
# stdout_value
# stderr_value

waiting for message


KeyboardInterrupt: 

In [ ]:
print(f'PARENT: {proc.pid} signaling child')
# sys.stdout.flush()
os.kill(proc.pid, signal.SIGUSR1)


PARENT: 1284069 signaling child


ProcessLookupError: [Errno 3] No such process

In [ ]:
stdout_raw, stderr_raw = proc.communicate()
stdout_value = stdout_raw.decode('utf-8')
stderr_value = stderr_raw.decode('utf-8')

print(f'stdout: {repr(stdout_value)}; stderr: {repr(stderr_value)}')
# [0].decode('utf-8')


stdout: ''; stderr: "python: can't open file '/home/n/devel/anting_remote/eos/tspace/candycan/nbs/../../candycan/send_message.py': [Errno 2] No such file or directory\n"


In [ ]:
datetime.fromtimestamp(message_proxy['timestamp']),db_can.decode_message(message_proxy['arbitration_id'],message_proxy['data'])

(datetime.datetime(2024, 2, 9, 13, 56, 32, 491982),
 {'Enable': 'Enabled', 'AverageRadius': 3.2, 'Temperature': 250.1})

In [ ]:
# close and remove vcan0
# !sshpass -v -p  asdf sudo ip link delete vcan0 

os.system("gpg -d -q ~/.sshpasswd.gpg | sshpass -v sudo ip link set down vcan0")


SSHPASS: searching for password prompt using match "assword"
SSHPASS: read: [sudo] password for n: 
SSHPASS: detected prompt. Sending password.
SSHPASS: read: 



0

In [ ]:
# delete vcan0
os.system("gpg -d -q ~/.sshpasswd.gpg | sshpass -v sudo ip link delete vcan0")

SSHPASS: searching for password prompt using match "assword"
SSHPASS: read: [sudo] password for n: 
SSHPASS: detected prompt. Sending password.
SSHPASS: read: 



0

In [ ]:
#| export
if __name__ == "__main__" and "__file__" in globals():   # in order to be compatible for both script and notebnook

    # print(os.getcwd())
    p = get_argparser()
    args = p.parse_args()
    # args = p.parse_args(
    #     [
    #         "--type",
    #         "socketcan",
    #         "--channel",
    #         "vcan0",
    #         "--bitrate",
    #         "250000",
    #         "--dbc",
    #         "./examples/motohawk_new.dbc",
    #         "--message",
    #         "ExampleMessage",
    #     ]
    # )
    # print(args)

    try:
        db: Database = cantools.database.load_file(args.dbc)
    except FileNotFoundError as e:
        print(f"File not found: {e}")


    payload = sys.stdin.buffer.read()
    
    send_msg(db=db,message=args.message,payload=payload,channel=args.channel,bitrate=args.bitrate,bus_type=args.type,is_extended=args.extended)


    signal.signal(signal.SIGUSR1, signal_usr1)
    # print("set message handler and sleep")
    sys.stdout.flush()
    now = time.time()
    done.wait()
    time_lapsed = time.time() - now
    print(f"Signal received after {time_lapsed:.3f} seconds")



In [ ]:
#| hide 
import nbdev; nbdev.nbdev_export()